**Basic Stock Forecaster using Yfinance API and LSTM model**

Install and Import Libraries

In [2]:
!pip install Yfinance
!pip install pandas
!pip install numpy
!pip install sckit-learn
!pip install matplotlib

ERROR: Could not find a version that satisfies the requirement sckit-learn (from versions: none)
ERROR: No matching distribution found for sckit-learn


In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Yfinance API test

In [24]:
data = yf.Ticker("SPY")
info = data.history(period='1mo', interval='1d')
info = info[["Open", "High", "Low", "Close", "Volume"]]

In [25]:
info.head()

,Open,High,Low,Close,Volume
Date,,,,,
2025-01-14 00:00:00-05:00,584.359985,585.000000,578.349976,582.190002,48420600
2025-01-15 00:00:00-05:00,590.330017,593.940002,589.200012,592.780029,56900200
2025-01-16 00:00:00-05:00,594.169983,594.349976,590.929993,591.640015,43319700
2025-01-17 00:00:00-05:00,596.960022,599.359985,595.609985,597.580017,58070600
2025-01-21 00:00:00-05:00,600.669983,603.059998,598.669983,603.049988,42532900


LSTM Model Structure

In [29]:
class StockLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=4, output_size=1):
        super(StockLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out